In [46]:
from __future__ import print_function # Python 2/3 compatibility
import boto3

In [47]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url='http://dynamodb.us-west-2.amazonaws.com')

In [48]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)
class MyEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, set):
          return list(o)
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

In [49]:
varTableName = 'GIS_ProductCatalog'

In [50]:
table = dynamodb.Table(varTableName)

try:
    table.delete()
except ClientError as e:
    print(e.response['Error']['Message'])

In [51]:
#dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")
table = dynamodb.create_table(
    TableName=varTableName,
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'Title',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'Title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

Table status: CREATING


In [52]:
# load data
import json
import decimal

#dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

#table = dynamodb.Table('GIS_ProductCatalog')

with open("ProductCatalog_data.json") as json_file:
    prods = json.load(json_file, parse_float = decimal.Decimal)
    for prod in prods:
        Id = int(prod['Id'])
        Title = prod['Title']
        ISBN = prod['ISBN']
        Authors = set(prod['Authors'])
        Price = decimal.Decimal(prod['Price'])
        Dim = prod['Dimensions']
        PageCount = int(prod['PageCount'])
        InPub = prod['InPublication']
        cate = prod['ProductCategory']
        reviews = prod['reviews'];
       
        print("Adding product catalog:", Id, Title)

        response = table.put_item(
           Item={
               'Id': Id,
               'Title': Title,
               'ISBN': ISBN,
               'Authors' : Authors,
               'Price': Price,
               'Dimensions': Dim,
               'PageCount' : PageCount,
               'InPublication' : InPub,
               'ProductCategory' : cate,
               'Reviews' : reviews
            }
        )
        print("PutItem succeeded:")
        print(json.dumps(response, indent=4, cls=DecimalEncoder))

Adding product catalog: 201 Book 101 Title
PutItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "35DQUJ2OLIUHHMR6QFNTPINNLFVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "35DQUJ2OLIUHHMR6QFNTPINNLFVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "2745614147",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "2",
            "date": "Wed, 08 Mar 2017 16:55:35 GMT"
        },
        "RetryAttempts": 0
    }
}
Adding product catalog: 102 Book 102 Title
PutItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "4BNLN4ECIDLPMQBOLAFRC3R9SBVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "4BNLN4ECIDLPMQBOLAFRC3R9SBVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "2745614147",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "2",
            

In [65]:
# get single item
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

#dynamodb = boto3.resource("dynamodb", region_name='us-west-2', endpoint_url="http://localhost:8000")

#table = dynamodb.Table('ProductCatalog')

Title = "Book 401 Title"
Id = 401

try:
    response = table.get_item(
        Key={
            'Id': Id,
            'Title': Title
        }
    )
    #print(response['Item'].Reviews.reviewEntries[0].reviewedOn)
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    #print(item['Reviews']['reviewEntries'][0]['reviewRank'])
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=MyEncoder))

GetItem succeeded:
{
    "Title": "Book 401 Title",
    "InPublication": true,
    "PageCount": 210,
    "Reviews": {
        "reviewEntries": [
            {
                "reviewedOn": "2017-02-25",
                "reviewer": "Imaya",
                "reviewRank": "FOUR",
                "reviewText": "Very good book"
            },
            {
                "reviewedOn": "2016-02-24",
                "reviewer": "Chitra",
                "reviewRank": "ONE",
                "reviewText": "Very bad book"
            }
        ]
    },
    "Dimensions": "8.5 x 11.0 x 0.2",
    "Authors": [
        "Imaya",
        "Karthik",
        "Chitra"
    ],
    "ISBN": "123-45678782",
    "Price": 19.99,
    "ProductCategory": "Book",
    "Id": 401
}


In [64]:
Title = "Book 401 Title"
Id = 401

# Conditional update (will fail)
print("Attempting conditional update...")

try:
    response = table.update_item(
    Key={
        'Id': Id,
        'Title': Title
        },        
        UpdateExpression="set Authors[2] = :a",
        ConditionExpression="size(Authors) >= :num",
        ExpressionAttributeValues={
            ':num': 2,
            ':a' : 'Chitra'
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("UpdateItem succeeded:")
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

Attempting conditional update...
UpdateItem succeeded:
{
    "Attributes": {
        "Authors": [
            "Chitra"
        ]
    },
    "ResponseMetadata": {
        "RequestId": "6ED3IRE6C5PJLS7FLTA495GL0RVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "6ED3IRE6C5PJLS7FLTA495GL0RVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "3600411371",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "49",
            "date": "Wed, 08 Mar 2017 17:01:38 GMT"
        },
        "RetryAttempts": 0
    }
}


In [58]:
response = table.query(
    KeyConditionExpression=Key('Id').eq(401)
)

for i in response['Items']:
    print(i['Id'], ":", i['Title'])

401 : Book 401 Title


In [59]:
response = table.query(
    ProjectionExpression="#pc, Title, ISBN, Price, Actors[0]",
    ExpressionAttributeNames={ "#pc": "ProductCategory" }, # Expression Attribute Names for Projection Expression only.
    KeyConditionExpression=Key('Id').eq(401) & Key('Title').between('B', 'F')
)

for i in response[u'Items']:
    print(json.dumps(i, cls=DecimalEncoder))

{"Price": 19.99, "Title": "Book 401 Title", "ProductCategory": "Book", "ISBN": "123-45678782"}


In [75]:
from decimal import *
fe = Key('Id').between(101, 405);
pe = "#pc, Title, ISBN, Price, Actors[0]"
# Expression Attribute Names for Projection Expression only.
ean = { "#pc": "ProductCategory" }
esk = None


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
    )

total = Decimal(0.0)
for i in response['Items']:
    total = total +  Price
    print(json.dumps(i, cls=DecimalEncoder))

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExpressionAttributeNames= ean,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )

    for i in response['Items']:
        total = total + Price
        print(json.dumps(i, cls=DecimalEncoder))
    
print('Total Price %f'%total)

{"Price": 19.99, "Title": "Book 401 Title", "ProductCategory": "Book", "ISBN": "123-45678782"}
{"Price": 2, "Title": "Book 101 Title", "ProductCategory": "Book", "ISBN": "111-1111111111"}
{"Price": 20, "Title": "Book 102 Title", "ProductCategory": "Book", "ISBN": "222-2222222222"}
Total Price 59.970000
